## Instalação e Importação

In [3]:
!apt-get install -y openjdk-17-jdk-headless ghostscript python3-tk
#!pip install tabula-py camelot-py[cv] pdfplumber pandas requests beautifulsoup4 openpyxl
%pip install tabula-py camelot-py[cv] pdfplumber pandas requests beautifulsoup4 openpyxl

'apt-get' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os, re, requests
from bs4 import BeautifulSoup
import pandas as pd
import tabula
from shape_to_files import shape_to_files

## Extrair pfs

In [2]:
OUT_DIR = "pdfs"
os.makedirs(OUT_DIR, exist_ok=True)

In [3]:
# URL do catálogo IBGE
BASE_CATALOG_URL = "https://biblioteca.ibge.gov.br/index.php/biblioteca-catalogo?view=detalhes&id=7236"

In [4]:
# 1) Pegar links de PDFs
resp = requests.get(BASE_CATALOG_URL)
soup = BeautifulSoup(resp.text, "html.parser")

pdf_links = []
for a in soup.find_all("a", href=True):
    href = a["href"]
    if ".pdf" in href and ("ipca" in href.lower() or "inpc" in href.lower()):
        if href.startswith("/"):
            href = "https://biblioteca.ibge.gov.br" + href
        pdf_links.append(href)

print("Total de PDFs encontrados:", len(pdf_links))

Total de PDFs encontrados: 377


In [14]:
pdf_links

['https://biblioteca.ibge.gov.br/visualizacao/periodicos/236/inpc_ipca_2025_jan.pdf',
 'https://biblioteca.ibge.gov.br/visualizacao/periodicos/236/inpc_ipca_2025_fev.pdf',
 'https://biblioteca.ibge.gov.br/visualizacao/periodicos/236/inpc_ipca_2025_mar.pdf',
 'https://biblioteca.ibge.gov.br/visualizacao/periodicos/236/inpc_ipca_2025_abr.pdf',
 'https://biblioteca.ibge.gov.br/visualizacao/periodicos/236/inpc_ipca_2025_maio.pdf',
 'https://biblioteca.ibge.gov.br/visualizacao/periodicos/236/inpc_ipca_2025_jun.pdf',
 'https://biblioteca.ibge.gov.br/visualizacao/periodicos/236/inpc_ipca_2025_jul.pdf',
 'https://biblioteca.ibge.gov.br/visualizacao/periodicos/236/inpc_ipca_2025_ago.pdf',
 'https://biblioteca.ibge.gov.br/visualizacao/periodicos/236/inpc_ipca_2025_set.pdf',
 'https://biblioteca.ibge.gov.br/visualizacao/periodicos/236/inpc_ipca_2024_jan.pdf',
 'https://biblioteca.ibge.gov.br/visualizacao/periodicos/236/inpc_ipca_2024_fev.pdf',
 'https://biblioteca.ibge.gov.br/visualizacao/periodi

In [5]:
# Filtrar pelos anos de intesse
years = ['2012', '2013', '2014', '2015', '2016','2017', '2018', '2019' , '2020']
pdf_links_2012_2020 = [link for link in pdf_links if any(year in link for year in years)]


## Baixar os arquivos

In [8]:
all_rows = []

In [9]:
# Percorrer os links
for url in pdf_links:
    # Verificar se o link contém algum dos anos desejados
    contem_ano_desejado = False
    for ano in years:
        if ano in url:
            contem_ano_desejado = True
            break

    # Se o ano não for desejado, pula para o próximo link
    if not contem_ano_desejado:
        continue

    # Extrair o nome do arquivo e montar o caminho de saída
    nome_arquivo = url.split("/")[-1]
    caminho_arquivo = os.path.join(OUT_DIR, nome_arquivo)

    # Verifica se o arquivo já foi baixado
    if not os.path.exists(caminho_arquivo):
        print(f"Baixando: {url}")
        resposta = requests.get(url)
        with open(caminho_arquivo, "wb") as f:
            f.write(resposta.content)
    else:
        print(f"Já existe: {nome_arquivo}")

Já existe: inpc_ipca_2020_jan.pdf
Já existe: inpc_ipca_2020_fev.pdf
Já existe: inpc_ipca_2020_mar.pdf
Já existe: inpc_ipca_2020_abr.pdf
Já existe: inpc_ipca_2020_maio.pdf
Já existe: inpc_ipca_2020_jun.pdf
Já existe: inpc_ipca_2020_jul.pdf
Já existe: inpc_ipca_2020_ago.pdf
Já existe: inpc_ipca_2020_set.pdf
Já existe: inpc_ipca_2020_out.pdf
Já existe: inpc_ipca_2020_nov.pdf
Já existe: inpc_ipca_2020_dez.pdf
Já existe: inpc_ipca_2019_jan.pdf
Já existe: inpc_ipca_2019_fev.pdf
Já existe: inpc_ipca_2019_mar.pdf
Já existe: inpc_ipca_2019_abr.pdf
Já existe: inpc_ipca_2019_maio.pdf
Já existe: inpc_ipca_2019_jun.pdf
Já existe: inpc_ipca_2019_jul.pdf
Já existe: inpc_ipca_2019_ago.pdf
Já existe: inpc_ipca_2019_set.pdf
Já existe: inpc_ipca_2019_out.pdf
Já existe: inpc_ipca_2019_nov.pdf
Já existe: inpc_ipca_2019_dez.pdf
Já existe: inpc_ipca_2018_jan.pdf
Já existe: inpc_ipca_2018_fev.pdf
Já existe: inpc_ipca_2018_mar.pdf
Já existe: inpc_ipca_2018_abr.pdf
Já existe: inpc_ipca_2018_maio.pdf
Já existe: 

## Extrair as tabelas

### fazendo um caso particular

In [10]:
url = pdf_links_2012_2020[0]
fname = os.path.join(OUT_DIR, url.split("/")[-1])
print(f"Lendo {fname}...")
r = requests.get(url)
with open(fname, "wb") as f:
    f.write(r.content)

Lendo pdfs\inpc_ipca_2020_jan.pdf...


In [11]:
pdf_links_2012_2020[0]

'https://biblioteca.ibge.gov.br/visualizacao/periodicos/236/inpc_ipca_2020_jan.pdf'

In [12]:
fname

'pdfs\\inpc_ipca_2020_jan.pdf'

In [49]:
#'pdfs\inpc_ipca_2020_mar.pdf'
#pdfs\inpc_ipca_2020_abr.pdf (12, 3)
#pdfs\inpc_ipca_2020_out.pdf (14, 3)
#pdfs\inpc_ipca_2018_abr.pdf (3,4)
# pdfs\inpc_ipca_2019_maio.pdf (12,4)
#pdfs\inpc_ipca_2017_fev.pdf (5,5)
#pdfs\inpc_ipca_2012_mar.pdf (2, 5)

fname = 'pdfs\\inpc_ipca_2019_maio.pdf'

In [50]:
# Extrair tabelas com Tabula
dfs = tabula.read_pdf(fname, pages="all", multiple_tables=True)

In [51]:
print("Número de tabelas extraídas:", len(dfs))
for i, df in enumerate(dfs[:2]):
    print("\n--- Tabela", i, "---")
    print(display(df))

Número de tabelas extraídas: 17

--- Tabela 0 ---


,Unnamed: 0,Unnamed: 1,Variação (%),Impacto (p.p.)
0,NaN,Grupo,NaN,NaN
1,NaN,NaN,Abril Maio,Abril Maio
2,Índice Geral,NaN,"0,57 0,13","0,57 0,13"
3,Alimentação e Bebidas,NaN,"0,63 -0,56","0,16 -0,14"
4,Habitação,NaN,"0,24 0,98","0,04 0,15"
5,Artigos de Residência,NaN,"-0,24 -0,10","-0,01 0,00"
6,Vestuário,NaN,"0,18 0,34","0,01 0,02"
7,Transportes,NaN,"0,94 0,07","0,17 0,01"
8,Saúde e Cuidados Pessoais,NaN,"1,51 0,59","0,18 0,07"
9,Despesas Pessoais,NaN,"0,17 0,16","0,02 0,02"


None

--- Tabela 1 ---


,"O resultado do grupo Alimentação e bebidas (-0,56%) deveu-se principalmente à queda de 0,89%\robservada no grupamento da alimentação no domicílio. O tomate, após apresentar alta de 28,64% em\rabril, caiu 15,08%, e o feijão-carioca acentuou a queda em relação ao mês anterior (passou de -9,09%\rpara -13,04%). As frutas (-2,87%) também apresentaram queda mais intensa em comparação com o mês\rde abril, quando registraram -0,71%. Por outro lado, o leite longa vida (2,37%) e a cenoura (15,74%)\rsubiram em maio, após apresentarem quedas de 0,30% e 0,07%, respectivamente, em abril.\r\rO grupo Habitação (0,98%), por sua vez, apresentou o maior impacto positivo no mês de maio\r(0,15 p.p.), influenciado principalmente pela alta de 2,18% no item energia elétrica. Todas as regiões\rpesquisadas apresentaram variações positivas, que foram desde o 0,23% na região metropolitana do Rio\rde Janeiro até os 7,32% registrados na região metropolitana de Fortaleza. De dezembro de 2018 a abril\rde 2019, havia vigorado a bandeira tarifária verde, em que não há cobrança adicional na conta de luz. Em\rmaio, passou a vigorar a bandeira tarifária amarela, com custo adicional de R$ 0,01 para cada quilowatt-\rhora consumido. Além disso, vários reajustes de tarifas foram incorporados, conforme mostra a tabela a\rseguir:\r\rEnergia elétrica\rRegiãoVariação (%)Reajuste (%)Data\rBelo Horizonte1,457,8928/05\rRecife5,375,5629/04\rFortaleza7,327,3922/04\rSalvador6,356,2122/04\rAracaju4,173,0422/04\rCampo Grande6,0512,3908/04\rRio de Janeiro*0,238,80 e 7,3001/04\r\r* No Rio de Janeiro, houve inicialmente um reajuste de 11,53% e 9,72% nas concessionárias, a partir de 15\rde março, sendo reduzido para 8,80% e 7,30%, respectivamente, a partir de 1o de abril.\r\rAinda em Habitação, a variação de 0,82% na taxa de água e esgoto reflete os reajustes de 4,72%\rna região metropolitana de São Paulo (3,15%), a partir de 11 de maio, e de 2,99% em Brasília (0,18%),\rvigente desde 1o de abril. A queda no gás encanado (-0,84%), por sua vez, é consequência da redução\rmédia de 1,40% nas tarifas residenciais praticadas na região metropolitana do Rio de Janeiro (-1,57%), a\rpartir de 1o de maio.\rO\rgásdebotijão,tambémdogrupoHabitação,apresentou-secomaltade1,35%,em\rdecorrência do reajuste médio de 3,43% autorizado pela Petrobras, nas refinarias, a partir de 05 de maio.\r\r- 9 -",Unnamed: 0,Unnamed: 1
0,Variação (%),Reajuste (%),Data
1,"1,45\r5,37\r7,32\r6,35\r4,17\r6,05\r0,23","7,89\r5,56\r7,39\r6,21\r3,04\r12,39\r8,80 e 7,30",NaN


None


In [16]:
# heurística: pegar a primeira tabela que tenha 'Grupo' e 'Impacto'
tbl = None
for df in dfs:
    if any("Variação" in str(c) for c in df.columns) and \
        any("Impacto" in str(c) for c in df.columns):
        tbl = df
        display(tbl)
        #break

,Unnamed: 0,Variação (%),Impacto (p.p.)
0,Grupo,NaN,NaN
1,NaN,Março Abril,Março Abril
2,Índice Geral,"0,07 -0,31","0,07 -0,31"
3,Alimentação e Bebidas,"1,13 1,79","0,22 0,35"
4,Habitação,"0,13 -0,10","0,02 -0,02"
5,Artigos de Residência,"-1,08 -1,37","-0,04 -0,05"
6,Vestuário,"0,21 0,10","0,01 0,00"
7,Transportes,"-0,90 -2,66","-0,18 -0,54"
8,Saúde e Cuidados Pessoais,"0,21 -0,22","0,03 -0,03"
9,Despesas Pessoais,"-0,23 -0,14","-0,03 -0,01"


In [17]:
tbl

,Unnamed: 0,Variação (%),Impacto (p.p.)
0,Grupo,NaN,NaN
1,NaN,Março Abril,Março Abril
2,Índice Geral,"0,07 -0,31","0,07 -0,31"
3,Alimentação e Bebidas,"1,13 1,79","0,22 0,35"
4,Habitação,"0,13 -0,10","0,02 -0,02"
5,Artigos de Residência,"-1,08 -1,37","-0,04 -0,05"
6,Vestuário,"0,21 0,10","0,01 0,00"
7,Transportes,"-0,90 -2,66","-0,18 -0,54"
8,Saúde e Cuidados Pessoais,"0,21 -0,22","0,03 -0,03"
9,Despesas Pessoais,"-0,23 -0,14","-0,03 -0,01"


In [18]:
tbl.loc[2]['Impacto (p.p.)']

'0,07 -0,31'

In [6]:
year_match = re.search(r'(\d{4})', fname)
month_match = re.search(r'_(jan|fev|mar|abr|mai|jun|jul|ago|set|out|nov|dez)', fname, re.I)


NameError: name 'fname' is not defined

In [20]:
month_match

<re.Match object; span=(19, 23), match='_abr'>

In [21]:
ano = year_match.group(1)
mes = month_match.group(1).lower().strip()
ano_mes = f"{ano}-{mes}"

In [22]:
tbl

,Unnamed: 0,Variação (%),Impacto (p.p.)
0,Grupo,NaN,NaN
1,NaN,Março Abril,Março Abril
2,Índice Geral,"0,07 -0,31","0,07 -0,31"
3,Alimentação e Bebidas,"1,13 1,79","0,22 0,35"
4,Habitação,"0,13 -0,10","0,02 -0,02"
5,Artigos de Residência,"-1,08 -1,37","-0,04 -0,05"
6,Vestuário,"0,21 0,10","0,01 0,00"
7,Transportes,"-0,90 -2,66","-0,18 -0,54"
8,Saúde e Cuidados Pessoais,"0,21 -0,22","0,03 -0,03"
9,Despesas Pessoais,"-0,23 -0,14","-0,03 -0,01"


In [23]:
tbl = tbl.drop([0, 1], axis=0).reset_index(drop=True)

In [24]:
# separar os valores da coluna de impacto por espaço (dezembro / janeiro)
impact_split = tbl["Impacto (p.p.)"].astype(str).str.split(" ", expand=True)

In [25]:
impact_split

,0,1
0,"0,07","-0,31"
1,"0,22","0,35"
2,"0,02","-0,02"
3,"-0,04","-0,05"
4,"0,01","0,00"
5,"-0,18","-0,54"
6,"0,03","-0,03"
7,"-0,03","-0,01"
8,"0,04","0,00"
9,"0,00","-0,01"


In [26]:
tbl["ano_mes"] = ano_mes

In [27]:
tbl['Impacto (p.p.)'] = impact_split[1]

In [28]:
tbl

,Unnamed: 0,Variação (%),Impacto (p.p.),ano_mes
0,Índice Geral,"0,07 -0,31","-0,31",2020-abr
1,Alimentação e Bebidas,"1,13 1,79","0,35",2020-abr
2,Habitação,"0,13 -0,10","-0,02",2020-abr
3,Artigos de Residência,"-1,08 -1,37","-0,05",2020-abr
4,Vestuário,"0,21 0,10","0,00",2020-abr
5,Transportes,"-0,90 -2,66","-0,54",2020-abr
6,Saúde e Cuidados Pessoais,"0,21 -0,22","-0,03",2020-abr
7,Despesas Pessoais,"-0,23 -0,14","-0,01",2020-abr
8,Educação,"0,59 0,00","0,00",2020-abr
9,Comunicação,"0,04 -0,20","-0,01",2020-abr


In [29]:
tbl = tbl.drop(columns = ['Variação (%)']).rename(columns = {'Unnamed: 0': 'Setor'})

In [30]:
all_rows = []


## Organizando as tabelas por shape

Cada um terá que ser tratado separadamente

In [9]:
shape_to_files

{(2, 4): ['pdfs\\inpc_ipca_2020_mar.pdf',
  'pdfs\\inpc_ipca_2020_jun.pdf',
  'pdfs\\inpc_ipca_2019_dez.pdf',
  'pdfs\\inpc_ipca_2018_jan.pdf',
  'pdfs\\inpc_ipca_2018_fev.pdf',
  'pdfs\\inpc_ipca_2018_mar.pdf',
  'pdfs\\inpc_ipca_2018_maio.pdf',
  'pdfs\\inpc_ipca_2018_jun.pdf',
  'pdfs\\inpc_ipca_2018_jul.pdf',
  'pdfs\\inpc_ipca_2018_ago.pdf',
  'pdfs\\inpc_ipca_2018_set.pdf',
  'pdfs\\inpc_ipca_2018_out.pdf',
  'pdfs\\inpc_ipca_2018_nov.pdf',
  'pdfs\\inpc_ipca_2018_dez.pdf',
  'pdfs\\inpc_ipca_2013_out.pdf'],
 (2, 5): ['pdfs\\inpc_ipca_2013_jan.pdf',
  'pdfs\\inpc_ipca_2012_mar.pdf',
  'pdfs\\inpc_ipca_2012_abr.pdf',
  'pdfs\\inpc_ipca_2012_jun.pdf',
  'pdfs\\inpc_ipca_2012_set.pdf',
  'pdfs\\inpc_ipca_2012_nov.pdf'],
 (3, 4): ['pdfs\\inpc_ipca_2018_abr.pdf',
  'pdfs\\inpc_ipca_2016_fev.pdf',
  'pdfs\\inpc_ipca_2016_jun.pdf',
  'pdfs\\inpc_ipca_2015_fev.pdf',
  'pdfs\\inpc_ipca_2015_set.pdf',
  'pdfs\\inpc_ipca_2015_dez.pdf'],
 (4, 4): ['pdfs\\inpc_ipca_2014_dez.pdf'],
 (4, 5): ['

In [ ]:
# from collections import defaultdict

# # Dicionário para armazenar os shapes e os arquivos associados
# shape_to_files = defaultdict(list)

# for url in pdf_links_2012_2020:
#     fname = os.path.join(OUT_DIR, url.split("/")[-1])
    
#     if not os.path.exists(fname):
#         r = requests.get(url)
#         with open(fname, "wb") as f:
#             f.write(r.content)

#     # Extrair tabelas
#     try:
#         dfs = tabula.read_pdf(fname, pages="all", multiple_tables=True, lattice=False)
#         #print(f"Extraído com lattice=False: {fname}")
#     except:
#         dfs = tabula.read_pdf(fname, pages="all", multiple_tables=True, lattice=True)

#     # Heurística: pegar a primeira tabela que tenha 'Variação' e 'Impacto'
#     tbl = None
#     for df in dfs:
#         if any("Variação" in str(c) for c in df.columns) and \
#            any("Impacto" in str(c) for c in df.columns):
#             tbl = df
#             break

#     if tbl is None:
#         print("Tabela não encontrada em:", fname)
#         continue

#     # Registrar o shape e o nome do arquivo
#     shape_to_files[tbl.shape].append(fname)

# # Exibir os resultados agrupados
# for shape, files in shape_to_files.items():
#     print(f"\nShape: {shape} → {len(files)} arquivos")
#     for f in files:
#         print("  ", f)

Tabela não encontrada em: pdfs\inpc_ipca_2020_jul.pdf
Tabela não encontrada em: pdfs\inpc_ipca_2012_jan.pdf

Shape: (12, 3) → 30 arquivos
   pdfs\inpc_ipca_2020_jan.pdf
   pdfs\inpc_ipca_2020_fev.pdf
   pdfs\inpc_ipca_2020_abr.pdf
   pdfs\inpc_ipca_2019_ago.pdf
   pdfs\inpc_ipca_2019_set.pdf
   pdfs\inpc_ipca_2019_out.pdf
   pdfs\inpc_ipca_2019_nov.pdf
   pdfs\inpc_ipca_2016_jan.pdf
   pdfs\inpc_ipca_2016_abr.pdf
   pdfs\inpc_ipca_2015_abr.pdf
   pdfs\inpc_ipca_2015_jun.pdf
   pdfs\inpc_ipca_2015_out.pdf
   pdfs\inpc_ipca_2014_jan.pdf
   pdfs\inpc_ipca_2014_fev.pdf
   pdfs\inpc_ipca_2014_mar.pdf
   pdfs\inpc_ipca_2014_abr.pdf
   pdfs\inpc_ipca_2013_fev.pdf
   pdfs\inpc_ipca_2013_mar.pdf
   pdfs\inpc_ipca_2013_abr.pdf
   pdfs\inpc_ipca_2013_maio.pdf
   pdfs\inpc_ipca_2013_jun.pdf
   pdfs\inpc_ipca_2013_jul.pdf
   pdfs\inpc_ipca_2013_ago.pdf
   pdfs\inpc_ipca_2013_set.pdf
   pdfs\inpc_ipca_2013_nov.pdf
   pdfs\inpc_ipca_2013_dez.pdf
   pdfs\inpc_ipca_2012_maio.pdf
   pdfs\inpc_ipca_2012_

In [ ]:
# shape_to_files
# # Salvar o dicionário em um arquivo Python  
# with open("shape_to_files.py", "w", encoding="utf-8") as f:
#     f.write("shape_to_files = ")
#     pprint.pprint(dict(shape_to_files), stream=f)

## Shape (12, 4)

In [22]:
arquivos_shape_12_4 = shape_to_files.get((12, 4), [])

In [23]:
tabelas_12_4 = []  # lista para armazenar cada tabela

In [24]:
for url in arquivos_shape_12_4:
    fname = url

    if not os.path.exists(fname):
        print(f"Arquivo não encontrado: {fname}")
        continue

    # extrair tabelas
    try:
        dfs = tabula.read_pdf(fname, pages="all", multiple_tables=True, lattice=False)
        print(f"Extraído com lattice=False: {fname}")
    except:
        dfs = tabula.read_pdf(fname, pages="all", multiple_tables=True, lattice=True)

    # heurística: pegar a primeira tabela que tenha 'Grupo' e 'Impacto'
    tbl = None
    for df in dfs:
        if any("Variação" in str(c) for c in df.columns) and \
            any("Impacto" in str(c) for c in df.columns):
            tbl = df
            break
    if tbl is None:
        print("Tabela não encontrada em:", fname)
        continue

    print(tbl.shape)
    #tbl = tbl.rename(columns=lambda x: str(x).strip().lower())
    tbl = tbl.rename(columns=lambda x: str(x).strip())
    year_match = re.search(r'(\d{4})', fname)
    month_match = re.search(r'_(jan|fev|mar|abr|mai|jun|jul|ago|set|out|nov|dez)', fname, re.I)

    ano = year_match.group(1)
    mes = month_match.group(1).lower().strip()
    ano_mes = f"{ano}-{mes}"

    tbl["ano_mes"] = ano_mes

    tbl = tbl.drop([0, 1], axis=0).reset_index(drop=True)


	
    tbl.columns.values[2] = 'Variação (%)'
    tbl.columns.values[3] = 'Impacto (p.p.)'

    # separar os valores da coluna de impacto por espaço (mes_anterior / mes_atual)
    variacao_split = tbl["Variação (%)"].astype(str).str.split(" ", expand=True)
    impact_split = tbl["Impacto (p.p.)"].astype(str).str.split(" ", expand=True)
    
    tbl['variação (%)'] = variacao_split[1]
    tbl['impacto (p.p.)'] = impact_split[1]
    tbl = tbl.drop(columns = [ 'Unnamed: 1']).rename(columns = {'Unnamed: 0': 'Setor'})

    display(tbl)

    tabelas_12_4.append(tbl)

df_final_12_4=pd.concat(tabelas_12_4, ignore_index=True)

Extraído com lattice=False: pdfs\inpc_ipca_2020_maio.pdf
(12, 4)


,Setor,Variação (%),Impacto (p.p.),ano_mes,variação (%),impacto (p.p.)
0,Índice Geral,"-0,31 -0,38","-0,31 -0,38",2020-mai,"-0,38","-0,38"
1,Alimentação e Bebidas,"1,79 0,24","0,35 0,05",2020-mai,"0,24","0,05"
2,Habitação,"-0,10 -0,25","-0,02 -0,04",2020-mai,"-0,25","-0,04"
3,Artigos de Residência,"-1,37 0,58","-0,05 0,02",2020-mai,"0,58","0,02"
4,Vestuário,"0,10 -0,58","0,00 -0,03",2020-mai,"-0,58","-0,03"
5,Transportes,"-2,66 -1,90","-0,54 -0,38",2020-mai,"-1,90","-0,38"
6,Saúde e Cuidados Pessoais,"-0,22 -0,10","-0,03 -0,01",2020-mai,"-0,10","-0,01"
7,Despesas Pessoais,"-0,14 -0,04","-0,01 0,00",2020-mai,"-0,04","0,00"
8,Educação,"0,00 0,02","0,00 0,00",2020-mai,"0,02","0,00"
9,Comunicação,"-0,20 0,24","-0,01 0,01",2020-mai,"0,24","0,01"


Extraído com lattice=False: pdfs\inpc_ipca_2019_jan.pdf
(12, 4)


,Setor,Variação (%),Impacto (p.p.),ano_mes,variação (%),impacto (p.p.)
0,Índice Geral,"0,15 0,32","0,15 0,32",2019-jan,"0,32","0,32"
1,Alimentação e Bebidas,"0,44 0,90","0,11 0,22",2019-jan,"0,90","0,22"
2,Habitação,"-0,15 0,24","-0,02 0,04",2019-jan,"0,24","0,04"
3,Artigos de Residência,"0,57 0,32","0,02 0,01",2019-jan,"0,32","0,01"
4,Vestuário,"1,14 -1,15","0,06 -0,07",2019-jan,"-1,15","-0,07"
5,Transportes,"-0,54 0,02","-0,10 0,01",2019-jan,"0,02","0,01"
6,Saúde e Cuidados Pessoais,"0,32 0,26","0,04 0,03",2019-jan,"0,26","0,03"
7,Despesas Pessoais,"0,29 0,61","0,03 0,07",2019-jan,"0,61","0,07"
8,Educação,"0,21 0,12","0,01 0,01",2019-jan,"0,12","0,01"
9,Comunicação,"0,01 0,04","0,00 0,00",2019-jan,"0,04","0,00"


Extraído com lattice=False: pdfs\inpc_ipca_2019_fev.pdf
(12, 4)


...

,Setor,Variação (%),Impacto (p.p.),ano_mes,variação (%),impacto (p.p.)
0,Índice Geral,"0,57 0,42","0,57 0,42",2014-out,"0,42","0,42"
1,Alimentação e Bebidas,"0,78 0,46","0,19 0,11",2014-out,"0,46","0,11"
2,Habitação,"0,77 0,68","0,11 0,10",2014-out,"0,68","0,10"
3,Artigos de Residência,"0,34 0,19","0,02 0,01",2014-out,"0,19","0,01"
4,Vestuário,"0,57 0,62","0,04 0,04",2014-out,"0,62","0,04"
5,Transportes,"0,63 0,39","0,12 0,07",2014-out,"0,39","0,07"
6,Saúde e Cuidados Pessoais,"0,33 0,39","0,04 0,04",2014-out,"0,39","0,04"
7,Despesas Pessoais,"0,39 0,36","0,04 0,04",2014-out,"0,36","0,04"
8,Educação,"0,18 0,11","0,01 0,01",2014-out,"0,11","0,01"
9,Comunicação,"0,13 -0,05","0,00 0,00",2014-out,"-0,05","0,00"


Extraído com lattice=False: pdfs\inpc_ipca_2014_nov.pdf
(12, 4)


,Setor,Variação (%),Impacto (p.p.),ano_mes,variação (%),impacto (p.p.)
0,Índice Geral,"0,42 0,51","0,42 0,51",2014-nov,"0,51","0,51"
1,Alimentação e Bebidas,"0,46 0,77","0,11 0,19",2014-nov,"0,77","0,19"
2,Habitação,"0,68 0,69","0,10 0,10",2014-nov,"0,69","0,10"
3,Artigos de Residência,"0,19 -0,04","0,01 0,00",2014-nov,"-0,04","0,00"
4,Vestuário,"0,62 0,39","0,04 0,03",2014-nov,"0,39","0,03"
5,Transportes,"0,39 0,43","0,07 0,08",2014-nov,"0,43","0,08"
6,Saúde e Cuidados Pessoais,"0,39 0,42","0,04 0,05",2014-nov,"0,42","0,05"
7,Despesas Pessoais,"0,36 0,48","0,04 0,05",2014-nov,"0,48","0,05"
8,Educação,"0,11 0,21","0,01 0,01",2014-nov,"0,21","0,01"
9,Comunicação,"-0,05 0,08","0,00 0,00",2014-nov,"0,08","0,00"


Extraído com lattice=False: pdfs\inpc_ipca_2012_fev.pdf
(12, 4)


,Setor,Variação (%),Impacto (p.p.),ano_mes,variação (%),impacto (p.p.)
0,Índice Geral,"0,56 0,45","0,56 0,45",2012-fev,"0,45","0,45"
1,Alimentação e Bebidas,"0,86 0,19","0,20 0,04",2012-fev,"0,19","0,04"
2,Habitação,"0,53 0,60","0,08 0,09",2012-fev,"0,60","0,09"
3,Artigos de Residência,"0,16 0,06","0,01 0,00",2012-fev,"0,06","0,00"
4,Vestuário,"0,07 -0,23","0,00 -0,02",2012-fev,"-0,23","-0,02"
5,Transportes,"0,69 -0,33","0,14 -0,07",2012-fev,"-0,33","-0,07"
6,Saúde e Cuidados Pessoais,"0,30 0,70","0,03 0,08",2012-fev,"0,70","0,08"
7,Despesas Pessoais,"0,71 0,88","0,07 0,09",2012-fev,"0,88","0,09"
8,Educação,"0,39 5,62","0,02 0,25",2012-fev,"5,62","0,25"
9,Comunicação,"0,21 -0,17","0,01 -0,01",2012-fev,"-0,17","-0,01"


In [28]:
df_final_12_4.to_excel("df_shapes/df_12_4.xlsx", index=False)

## Shape (12,3)

In [25]:
arquivos_shape_12_3 = shape_to_files.get((12, 3), [])

In [26]:
tabelas_12_3 = []  # lista para armazenar cada tabela


In [27]:
for url in arquivos_shape_12_3:
    fname = url

    if not os.path.exists(fname):
        print(f"Arquivo não encontrado: {fname}")
        continue

    # extrair tabelas
    try:
        dfs = tabula.read_pdf(fname, pages="all", multiple_tables=True, lattice=False)
        print(f"Extraído com lattice=False: {fname}")
    except:
        dfs = tabula.read_pdf(fname, pages="all", multiple_tables=True, lattice=True)

    # heurística: pegar a primeira tabela que tenha 'Grupo' e 'Impacto'
    tbl = None
    for df in dfs:
        if any("Variação" in str(c) for c in df.columns) and \
            any("Impacto" in str(c) for c in df.columns):
            tbl = df
            break
    if tbl is None:
        print("Tabela não encontrada em:", fname)
        continue

    print(tbl.shape)
    tbl = tbl.rename(columns=lambda x: str(x).strip())
    
    year_match = re.search(r'(\d{4})', fname)
    month_match = re.search(r'_(jan|fev|mar|abr|mai|jun|jul|ago|set|out|nov|dez)', fname, re.I)

    ano = year_match.group(1)
    mes = month_match.group(1).lower().strip()
    ano_mes = f"{ano}-{mes}"

    tbl["ano_mes"] = ano_mes

    tbl = tbl.drop([0, 1], axis=0).reset_index(drop=True)
    
    tbl.columns.values[1] = 'Variação (%)'
    tbl.columns.values[2] = 'Impacto (p.p.)'


    # separar os valores da coluna de impacto por espaço (mes_anterior / mes_atual)
    impact_split = tbl["Impacto (p.p.)"].astype(str).str.split(" ", expand=True)
    variacao_split = tbl["Variação (%)"].astype(str).str.split(" ", expand=True)

    tbl['variação (%)'] = variacao_split[1]
    tbl['impacto (p.p.)'] = impact_split[1]
    
    tbl = tbl.rename(columns = {'Unnamed: 0': 'Setor'})

    display(tbl)

    tabelas_12_3.append(tbl)

df_final_12_3=pd.concat(tabelas_12_3, ignore_index=True)



Extraído com lattice=False: pdfs\inpc_ipca_2020_jan.pdf
(12, 3)


,Setor,Variação (%),Impacto (p.p.),ano_mes,variação (%),impacto (p.p.)
0,Índice Geral,"1,15 0,21","1,15 0,21",2020-jan,"0,21","0,21"
1,Alimentação e Bebidas,"3,38 0,39","0,83 0,07",2020-jan,"0,39","0,07"
2,Habitação,"-0,82 0,55","-0,13 0,08",2020-jan,"0,55","0,08"
3,Artigos de Residência,"-0,48 -0,07","-0,02 0,00",2020-jan,"-0,07","0,00"
4,Vestuário,"0,00 -0,48","0,00 -0,02",2020-jan,"-0,48","-0,02"
5,Transportes,"1,54 0,32","0,28 0,06",2020-jan,"0,32","0,06"
6,Saúde e Cuidados Pessoais,"0,42 -0,32","0,05 -0,04",2020-jan,"-0,32","-0,04"
7,Despesas Pessoais,"0,92 0,35","0,10 0,04",2020-jan,"0,35","0,04"
8,Educação,"0,20 0,16","0,01 0,01",2020-jan,"0,16","0,01"
9,Comunicação,"0,66 0,12","0,03 0,01",2020-jan,"0,12","0,01"


Extraído com lattice=False: pdfs\inpc_ipca_2020_fev.pdf
(12, 3)


,Setor,Variação (%),Impacto (p.p.),ano_mes,variação (%),impacto (p.p.)
0,Índice Geral,"0,21 0,25","0,21 0,25",2020-fev,"0,25","0,25"
1,Alimentação e Bebidas,"0,39 0,11","0,07 0,02",2020-fev,"0,11","0,02"
2,Habitação,"0,55 -0,39","0,08 -0,06",2020-fev,"-0,39","-0,06"
3,Artigos de Residência,"-0,07 -0,08","0,00 0,00",2020-fev,"-0,08","0,00"
4,Vestuário,"-0,48 -0,73","-0,02 -0,03",2020-fev,"-0,73","-0,03"
5,Transportes,"0,32 -0,23","0,06 -0,05",2020-fev,"-0,23","-0,05"
6,Saúde e Cuidados Pessoais,"-0,32 0,73","-0,04 0,10",2020-fev,"0,73","0,10"
7,Despesas Pessoais,"0,35 0,31","0,04 0,03",2020-fev,"0,31","0,03"
8,Educação,"0,16 3,70","0,01 0,23",2020-fev,"3,70","0,23"
9,Comunicação,"0,12 0,21","0,01 0,01",2020-fev,"0,21","0,01"


Extraído com lattice=False: pdfs\inpc_ipca_2020_abr.pdf
(12, 3)


...

,Setor,Variação (%),Impacto (p.p.),ano_mes,variação (%),impacto (p.p.)
0,Índice Geral,"0,43 0,41","0,43 0,41",2012-ago,"0,41","0,41"
1,Alimentação e Bebidas,"0,91 0,88","0,21 0,21",2012-ago,"0,88","0,21"
2,Habitação,"0,54 0,22","0,08 0,03",2012-ago,"0,22","0,03"
3,Artigos de Residência,"-0,01 0,40","0,00 0,02",2012-ago,"0,40","0,02"
4,Vestuário,"0,04 0,19","0,00 0,01",2012-ago,"0,19","0,01"
5,Transportes,"-0,03 0,06","-0,01 0,01",2012-ago,"0,06","0,01"
6,Saúde e Cuidados Pessoais,"0,36 0,53","0,04 0,06",2012-ago,"0,53","0,06"
7,Despesas Pessoais,"0,91 0,42","0,09 0,04",2012-ago,"0,42","0,04"
8,Educação,"0,12 0,51","0,01 0,02",2012-ago,"0,51","0,02"
9,Comunicação,"0,15 -0,01","0,01 0,00",2012-ago,"-0,01","0,00"


Extraído com lattice=False: pdfs\inpc_ipca_2012_out.pdf
(12, 3)


,Setor,Variação (%),Impacto (p.p.),ano_mes,variação (%),impacto (p.p.)
0,Índice Geral,"0,57 0,59","0,57 0,59",2012-out,"0,59","0,59"
1,Alimentação e Bebidas,"1,26 1,36","0,30 0,32",2012-out,"1,36","0,32"
2,Habitação,"0,71 0,38","0,10 0,06",2012-out,"0,38","0,06"
3,Artigos de Residência,"0,18 0,37","0,01 0,02",2012-out,"0,37","0,02"
4,Vestuário,"0,89 1,09","0,06 0,07",2012-out,"1,09","0,07"
5,Transportes,"-0,08 0,24","-0,02 0,05",2012-out,"0,24","0,05"
6,Saúde e Cuidados Pessoais,"0,32 0,48","0,04 0,05",2012-out,"0,48","0,05"
7,Despesas Pessoais,"0,73 0,10","0,08 0,01",2012-out,"0,10","0,01"
8,Educação,"0,10 0,05","0,00 0,00",2012-out,"0,05","0,00"
9,Comunicação,"0,03 0,31","0,00 0,01",2012-out,"0,31","0,01"


Extraído com lattice=False: pdfs\inpc_ipca_2012_dez.pdf
(12, 3)


,Setor,Variação (%),Impacto (p.p.),ano_mes,variação (%),impacto (p.p.)
0,Índice Geral,"0,60 0,79","0,60 0,79",2012-dez,"0,79","0,79"
1,Alimentação e Bebidas,"0,79 1,03","0,19 0,25",2012-dez,"1,03","0,25"
2,Habitação,"0,64 0,63","0,09 0,09",2012-dez,"0,63","0,09"
3,Artigos de Residência,"0,47 0,27","0,02 0,01",2012-dez,"0,27","0,01"
4,Vestuário,"0,86 1,11","0,06 0,07",2012-dez,"1,11","0,07"
5,Transportes,"0,68 0,75","0,13 0,15",2012-dez,"0,75","0,15"
6,Saúde e Cuidados Pessoais,"0,32 0,40","0,04 0,05",2012-dez,"0,40","0,05"
7,Despesas Pessoais,"0,53 1,60","0,06 0,16",2012-dez,"1,60","0,16"
8,Educação,"0,05 0,19","0,00 0,01",2012-dez,"0,19","0,01"
9,Comunicação,"0,31 0,03","0,01 0,00",2012-dez,"0,03","0,00"


In [29]:
df_final_12_3.to_excel("df_shapes/df_12_3.xlsx", index=False)

## Shape (2,5)

In [30]:
arquivos_shape_2_5 = shape_to_files.get((2,5), [])

In [31]:
tabelas_2_5 = []  

In [32]:
for url in arquivos_shape_2_5:
    fname = url

    if not os.path.exists(fname):
        print(f"Arquivo não encontrado: {fname}")
        continue

    # extrair tabelas
    try:
        dfs = tabula.read_pdf(fname, pages="all", multiple_tables=True, lattice=False)
        print(f"Extraído com lattice=False: {fname}")
    except:
        dfs = tabula.read_pdf(fname, pages="all", multiple_tables=True, lattice=True)

    # heurística: pegar a primeira tabela que tenha 'Grupo' e 'Impacto'
    tbl = None
    for df in dfs:
        if any("Variação" in str(c) for c in df.columns) and \
            any("Impacto" in str(c) for c in df.columns):
            tbl = df
            break
    if tbl is None:
        print("Tabela não encontrada em:", fname)
        continue

    print(tbl.shape)

    tbl = tbl.rename(columns=lambda x: str(x).strip())

    year_match = re.search(r'(\d{4})', fname)
    month_match = re.search(r'_(jan|fev|mar|abr|mai|jun|jul|ago|set|out|nov|dez)', fname, re.I)

    ano = year_match.group(1)
    mes = month_match.group(1).lower().strip()
    ano_mes = f"{ano}-{mes}"

    tbl["ano_mes"] = ano_mes

    tbl = tbl.drop([0], axis=0).reset_index(drop=True)
    tbl = tbl.drop(['Variação (%)', 'Unnamed: 0'], axis=1)


    tbl.columns.values[1] = 'Variação (%)'
    tbl.columns.values[2] = 'Impacto (p.p.)'

    cols_to_split = ['Grupo', 'Variação (%)', 'Impacto (p.p.)']
    tbl[cols_to_split] = tbl[cols_to_split].apply(lambda x: x.str.split('\r'))
    tbl = tbl.explode(cols_to_split, ignore_index=True)
    
    #tbl = tbl.drop(columns = ['Variação (%)']).rename(columns = {'Unnamed: 0': 'Setor'})

    tbl = tbl.rename(columns={'Grupo': 'Setor'})
    
    display(tbl)

    tabelas_2_5.append(tbl)

df_final_2_5=pd.concat(tabelas_2_5, ignore_index=True)



Extraído com lattice=False: pdfs\inpc_ipca_2013_jan.pdf
(2, 5)


,Setor,Variação (%),Impacto (p.p.),ano_mes
0,Índice Geral,"0,86","0,86",2013-jan
1,Alimentação e Bebidas,"1,99","0,48",2013-jan
2,Habitação,"-0,20","-0,03",2013-jan
3,Artigos de Residência,"1,15","0,05",2013-jan
4,Vestuário,"-0,53","-0,04",2013-jan
5,Transportes,"0,75","0,15",2013-jan
6,Saúde e Cuidados Pessoais,"0,73","0,08",2013-jan
7,Despesas Pessoais,"1,55","0,16",2013-jan
8,Educação,"0,35","0,02",2013-jan
9,Comunicação,"-0,08","0,00",2013-jan


Extraído com lattice=False: pdfs\inpc_ipca_2012_mar.pdf
(2, 5)


,Setor,Variação (%),Impacto (p.p.),ano_mes
0,Índice Geral,"0,21","0,21",2012-mar
1,Alimentação e Bebidas,"0,25","0,06",2012-mar
2,Habitação,"0,48","0,07",2012-mar
3,Artigos de Residência,"-0,40","-0,02",2012-mar
4,Vestuário,"-0,61","-0,04",2012-mar
5,Transportes,"0,16","0,03",2012-mar
6,Saúde e Cuidados Pessoais,"0,38","0,04",2012-mar
7,Despesas Pessoais,"0,55","0,06",2012-mar
8,Educação,"0,54","0,02",2012-mar
9,Comunicação,"-0,36","-0,02",2012-mar


Extraído com lattice=False: pdfs\inpc_ipca_2012_abr.pdf
(2, 5)


...

,Setor,Variação (%),Impacto (p.p.),ano_mes
0,Índice Geral,"0,08","0,08",2012-jun
1,Alimentação e Bebidas,"0,68","0,16",2012-jun
2,Habitação,"0,28","0,04",2012-jun
3,Artigos de Residência,"-0,03","0,00",2012-jun
4,Vestuário,"0,39","0,03",2012-jun
5,Transportes,"-1,18","-0,24",2012-jun
6,Saúde e Cuidados Pessoais,"0,38","0,04",2012-jun
7,Despesas Pessoais,"0,47","0,05",2012-jun
8,Educação,"0,06","0,00",2012-jun
9,Comunicação,"-0,01","0,00",2012-jun


Extraído com lattice=False: pdfs\inpc_ipca_2012_set.pdf
(2, 5)


,Setor,Variação (%),Impacto (p.p.),ano_mes
0,Índice Geral,"0,57","0,57",2012-set
1,Alimentação e Bebidas,"1,26","0,30",2012-set
2,Habitação,"0,71","0,10",2012-set
3,Artigos de Residência,"0,18","0,01",2012-set
4,Vestuário,"0,89","0,06",2012-set
5,Transportes,"-0,08","-0,02",2012-set
6,Saúde e Cuidados Pessoais,"0,32","0,04",2012-set
7,Despesas Pessoais,"0,73","0,08",2012-set
8,Educação,"0,10","0,00",2012-set
9,Comunicação,"0,03","0,00",2012-set


Extraído com lattice=False: pdfs\inpc_ipca_2012_nov.pdf
(2, 5)


,Setor,Variação (%),Impacto (p.p.),ano_mes
0,Índice Geral,"0,60","0,60",2012-nov
1,Alimentação e Bebidas,"0,79","0,19",2012-nov
2,Habitação,"0,64","0,09",2012-nov
3,Artigos de Residência,"0,47","0,02",2012-nov
4,Vestuário,"0,86","0,06",2012-nov
5,Transportes,"0,68","0,13",2012-nov
6,Saúde e Cuidados Pessoais,"0,32","0,04",2012-nov
7,Despesas Pessoais,"0,53","0,05",2012-nov
8,Educação,"0,05","0,00",2012-nov
9,Comunicação,"0,31","0,01",2012-nov


In [89]:
df_final_2_5

,Setor,Variação (%),Impacto (p.p.),ano_mes
0,Índice Geral,"0,86","0,86",2013-jan
1,Alimentação e Bebidas,"1,99","0,48",2013-jan
2,Habitação,"-0,20","-0,03",2013-jan
3,Artigos de Residência,"1,15","0,05",2013-jan
4,Vestuário,"-0,53","-0,04",2013-jan
5,Transportes,"0,75","0,15",2013-jan
6,Saúde e Cuidados Pessoais,"0,73","0,08",2013-jan
7,Despesas Pessoais,"1,55","0,16",2013-jan
8,Educação,"0,35","0,02",2013-jan
9,Comunicação,"-0,08","0,00",2013-jan


In [43]:
df_final_2_5.to_excel("df_shapes/df_2_5.xlsx", index=False)

## Shape (14,3)

In [34]:
arquivos_shape_14_3 = shape_to_files.get((14,3), [])

In [35]:
tabelas_14_3 = []

In [36]:
for url in arquivos_shape_14_3:
    fname = url

    if not os.path.exists(fname):
        print(f"Arquivo não encontrado: {fname}")
        continue

    # extrair tabelas
    try:
        dfs = tabula.read_pdf(fname, pages="all", multiple_tables=True, lattice=False)
        print(f"Extraído com lattice=False: {fname}")
    except:
        dfs = tabula.read_pdf(fname, pages="all", multiple_tables=True, lattice=True)

    # heurística: pegar a primeira tabela que tenha 'Grupo' e 'Impacto'
    tbl = None
    for df in dfs:
        if any("Variação" in str(c) for c in df.columns) and \
            any("Impacto" in str(c) for c in df.columns):
            tbl = df
            break
    if tbl is None:
        print("Tabela não encontrada em:", fname)
        continue

    print(tbl.shape)
    tbl = tbl.rename(columns=lambda x: str(x).strip())
    
    year_match = re.search(r'(\d{4})', fname)
    month_match = re.search(r'_(jan|fev|mar|abr|mai|jun|jul|ago|set|out|nov|dez)', fname, re.I)

    ano = year_match.group(1)
    mes = month_match.group(1).lower().strip()
    ano_mes = f"{ano}-{mes}"

    tbl["ano_mes"] = ano_mes

    tbl = tbl.drop([0, 1,2,4], axis=0).reset_index(drop=True)
    
    tbl.columns.values[1] = 'Variação (%)'
    tbl.columns.values[2] = 'Impacto (p.p.)'


    # separar os valores da coluna de impacto por espaço (mes_anterior / mes_atual)
    impact_split = tbl["Impacto (p.p.)"].astype(str).str.split(" ", expand=True)
    variacao_split = tbl["Variação (%)"].astype(str).str.split(" ", expand=True)

    tbl['variação (%)'] = variacao_split[1]
    tbl['impacto (p.p.)'] = impact_split[1]
    
    #tbl = tbl.drop(columns = ['Variação (%)']).rename(columns = {'Unnamed: 0': 'Setor'})
    tbl = tbl.rename(columns = {'Unnamed: 0': 'Setor'})

    display(tbl)

    tabelas_14_3.append(tbl)

df_final_14_3=pd.concat(tabelas_14_3, ignore_index=True)



Extraído com lattice=False: pdfs\inpc_ipca_2020_ago.pdf
(14, 3)


,Setor,Variação (%),Impacto (p.p.),ano_mes,variação (%),impacto (p.p.)
0,Índice Geral,"0,36 0,24","0,36 0,24",2020-ago,"0,24","0,24"
1,Alimentação e bebidas,"0,01 0,78","0,00 0,15",2020-ago,"0,78","0,15"
2,Habitação,"0,80 0,36","0,13 0,05",2020-ago,"0,36","0,05"
3,Artigos de residência,"0,90 0,56","0,03 0,02",2020-ago,"0,56","0,02"
4,Vestuário,"-0,52 -0,78","-0,02 -0,03",2020-ago,"-0,78","-0,03"
5,Transportes,"0,78 0,82","0,15 0,16",2020-ago,"0,82","0,16"
6,Saúde e cuidados pessoais,"0,44 0,50","0,06 0,07",2020-ago,"0,50","0,07"
7,Despesas pessoais,"-0,11 -0,01","-0,01 0,00",2020-ago,"-0,01","0,00"
8,Educação,"-0,12 -3,47","-0,01 -0,22",2020-ago,"-3,47","-0,22"
9,Comunicação,"0,51 0,67","0,03 0,04",2020-ago,"0,67","0,04"


Extraído com lattice=False: pdfs\inpc_ipca_2020_set.pdf
(14, 3)


,Setor,Variação (%),Impacto (p.p.),ano_mes,variação (%),impacto (p.p.)
0,Índice Geral,"0,24 0,64","0,24 0,64",2020-set,"0,64","0,64"
1,Alimentação e bebidas,"0,78 2,28","0,15 0,46",2020-set,"2,28","0,46"
2,Habitação,"0,36 0,37","0,05 0,06",2020-set,"0,37","0,06"
3,Artigos de residência,"0,56 1,00","0,02 0,04",2020-set,"1,00","0,04"
4,Vestuário,"-0,78 0,37","-0,03 0,02",2020-set,"0,37","0,02"
5,Transportes,"0,82 0,70","0,16 0,14",2020-set,"0,70","0,14"
6,Saúde e cuidados pessoais,"0,50 -0,64","0,07 -0,09",2020-set,"-0,64","-0,09"
7,Despesas pessoais,"-0,01 0,09","0,00 0,01",2020-set,"0,09","0,01"
8,Educação,"-3,47 -0,09","-0,22 -0,01",2020-set,"-0,09","-0,01"
9,Comunicação,"0,67 0,15","0,04 0,01",2020-set,"0,15","0,01"


Extraído com lattice=False: pdfs\inpc_ipca_2020_out.pdf
(14, 3)


,Setor,Variação (%),Impacto (p.p.),ano_mes,variação (%),impacto (p.p.)
0,Índice Geral,"0,64 0,86","0,64 0,86",2020-out,"0,86","0,86"
1,Alimentação e bebidas,"2,28 1,93","0,46 0,39",2020-out,"1,93","0,39"
2,Habitação,"0,37 0,36","0,06 0,05",2020-out,"0,36","0,05"
3,Artigos de residência,"1,00 1,53","0,04 0,06",2020-out,"1,53","0,06"
4,Vestuário,"0,37 1,11","0,02 0,05",2020-out,"1,11","0,05"
5,Transportes,"0,70 1,19","0,14 0,24",2020-out,"1,19","0,24"
6,Saúde e cuidados pessoais,"-0,64 0,28","-0,09 0,04",2020-out,"0,28","0,04"
7,Despesas pessoais,"0,09 0,19","0,01 0,02",2020-out,"0,19","0,02"
8,Educação,"-0,09 -0,04","-0,01 0,00",2020-out,"-0,04","0,00"
9,Comunicação,"0,15 0,21","0,01 0,01",2020-out,"0,21","0,01"


Extraído com lattice=False: pdfs\inpc_ipca_2020_nov.pdf
(14, 3)


,Setor,Variação (%),Impacto (p.p.),ano_mes,variação (%),impacto (p.p.)
0,Índice Geral,"0,86 0,89","0,86 0,89",2020-nov,"0,89","0,89"
1,Alimentação e bebidas,"1,93 2,54","0,39 0,53",2020-nov,"2,54","0,53"
2,Habitação,"0,36 0,44","0,05 0,07",2020-nov,"0,44","0,07"
3,Artigos de residência,"1,53 0,86","0,06 0,03",2020-nov,"0,86","0,03"
4,Vestuário,"1,11 0,07","0,05 0,00",2020-nov,"0,07","0,00"
5,Transportes,"1,19 1,33","0,24 0,26",2020-nov,"1,33","0,26"
6,Saúde e cuidados pessoais,"0,28 -0,13","0,04 -0,02",2020-nov,"-0,13","-0,02"
7,Despesas pessoais,"0,19 0,01","0,02 0,00",2020-nov,"0,01","0,00"
8,Educação,"-0,04 -0,02","0,00 0,00",2020-nov,"-0,02","0,00"
9,Comunicação,"0,21 0,29","0,01 0,02",2020-nov,"0,29","0,02"


Extraído com lattice=False: pdfs\inpc_ipca_2020_dez.pdf
(14, 3)


,Setor,Variação (%),Impacto (p.p.),ano_mes,variação (%),impacto (p.p.)
0,Índice Geral,"0,89 1,35","0,89 1,35",2020-dez,"1,35","1,35"
1,Alimentação e bebidas,"2,54 1,74","0,53 0,36",2020-dez,"1,74","0,36"
2,Habitação,"0,44 2,88","0,07 0,45",2020-dez,"2,88","0,45"
3,Artigos de residência,"0,86 1,76","0,03 0,07",2020-dez,"1,76","0,07"
4,Vestuário,"0,07 0,59","0,00 0,03",2020-dez,"0,59","0,03"
5,Transportes,"1,33 1,36","0,26 0,27",2020-dez,"1,36","0,27"
6,Saúde e cuidados pessoais,"-0,13 0,40","-0,02 0,05",2020-dez,"0,40","0,05"
7,Despesas pessoais,"0,01 0,65","0,00 0,07",2020-dez,"0,65","0,07"
8,Educação,"-0,02 0,48","0,00 0,03",2020-dez,"0,48","0,03"
9,Comunicação,"0,29 0,39","0,02 0,02",2020-dez,"0,39","0,02"


In [37]:
df_final_14_3.to_excel("df_shapes/df_14_3.xlsx", index=False)

## Shape (12,8)

In [38]:
arquivos_shape_12_8 = shape_to_files.get((12,8), [])


In [39]:
arquivos_shape_12_8 = shape_to_files.get((12,8), [])

fname = arquivos_shape_12_8[0]

dfs = tabula.read_pdf(fname, pages="all", multiple_tables=True)

print("Número de tabelas extraídas:", len(dfs))
for i, df in enumerate(dfs[:2]):
    print("\n--- Tabela", i, "---")
    #teste = df
    display(df)

Número de tabelas extraídas: 33

--- Tabela 0 ---


,Unnamed: 0,Variação no,Reajuste
0,Área,mês (%),extraordinário
1,NaN,NaN,(%)
2,Campo Grande,"34,77","26,67"
3,Curitiba,"32,73","31,86"
4,Porto Alegre,"27,21","26,31"
5,Goiânia,"27,14","25,12"
6,Fortaleza,"25,81","8,91"
7,São Paulo,"25,63","27,91"
8,Brasília,"23,85","21,98"
9,Belo Horizonte,"23,61","21,39"



--- Tabela 1 ---


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Variação (%),Unnamed: 3,Unnamed: 4,Impacto (p.p.),Unnamed: 5
0,NaN,Grupo,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,Janeiro,Fevereiro,Março,Janeiro,Fevereiro,Março
2,Índice Geral,NaN,"1,24","1,22","1,32","1,24","1,22","1,32"
3,Alimentação e Bebidas,NaN,"1,48","0,81","1,17","0,37","0,20","0,29"
4,Habitação,NaN,"2,42","1,22","5,29","0,35","0,18","0,79"
5,Artigos de Residência,NaN,"-0,28","0,87","0,35","-0,01","0,04","0,01"
6,Vestuário,NaN,"-0,69","-0,60","0,59","-0,04","-0,04","0,04"
7,Transportes,NaN,"1,83","2,20","0,46","0,34","0,41","0,09"
8,Saúde e Cuidados Pessoais,NaN,"0,32","0,60","0,69","0,03","0,07","0,08"
9,Despesas Pessoais,NaN,"1,68","0,86","0,36","0,18","0,09","0,04"


In [16]:
arquivos_shape_12_8

['pdfs\\inpc_ipca_2015_mar.pdf']

In [40]:
tabelas_12_8 = []

In [41]:
for url in arquivos_shape_12_8:
    fname = url

    if not os.path.exists(fname):
        print(f"Arquivo não encontrado: {fname}")
        continue

    # extrair tabelas
    try:
        dfs = tabula.read_pdf(fname, pages="all", multiple_tables=True, lattice=False)
        print(f"Extraído com lattice=False: {fname}")
    except:
        dfs = tabula.read_pdf(fname, pages="all", multiple_tables=True, lattice=True)

    # heurística: pegar a primeira tabela que tenha 'Grupo' e 'Impacto'
    tbl = None
    for df in dfs:
        if any("Variação" in str(c) for c in df.columns) and \
            any("Impacto" in str(c) for c in df.columns):
            tbl = df
            break
    if tbl is None:
        print("Tabela não encontrada em:", fname)
        continue

    print(tbl.shape)
    tbl = tbl.rename(columns=lambda x: str(x).strip())
    
    year_match = re.search(r'(\d{4})', fname)
    month_match = re.search(r'_(jan|fev|mar|abr|mai|jun|jul|ago|set|out|nov|dez)', fname, re.I)

    ano = year_match.group(1)
    mes = month_match.group(1).lower().strip()
    ano_mes = f"{ano}-{mes}"

    tbl["ano_mes"] = ano_mes

    tbl = tbl.drop([0, 1], axis=0).reset_index(drop=True)
    
    tbl = tbl.drop(columns = ['Unnamed: 1','Unnamed: 2','Variação (%)', 'Unnamed: 4','Impacto (p.p.)']).rename(columns = {'Unnamed: 0': 'Setor'})
    
    tbl.columns.values[1] = 'variação (%)'
    tbl.columns.values[2] = 'impacto (p.p.)'


    # separar os valores da coluna de impacto por espaço (mes_anterior / mes_atual)
    #impact_split = tbl["Impacto (p.p.)"].astype(str).str.split(" ", expand=True)
    #variacao_split = tbl["Variação (%)"].astype(str).str.split(" ", expand=True)

    #tbl['variação (%)'] = variacao_split[1]
    #tbl['impacto'] = impact_split[1]
    
    #tbl = tbl.drop(columns = ['Variação (%)']).rename(columns = {'Unnamed: 0': 'Setor'})

    display(tbl)

    tabelas_12_8.append(tbl)

df_final_12_8=pd.concat(tabelas_12_8, ignore_index=True)



Extraído com lattice=False: pdfs\inpc_ipca_2015_mar.pdf
(12, 8)


,Setor,variação (%),impacto (p.p.),ano_mes
0,Índice Geral,"1,32","1,32",2015-mar
1,Alimentação e Bebidas,"1,17","0,29",2015-mar
2,Habitação,"5,29","0,79",2015-mar
3,Artigos de Residência,"0,35","0,01",2015-mar
4,Vestuário,"0,59","0,04",2015-mar
5,Transportes,"0,46","0,09",2015-mar
6,Saúde e Cuidados Pessoais,"0,69","0,08",2015-mar
7,Despesas Pessoais,"0,36","0,04",2015-mar
8,Educação,"0,75","0,03",2015-mar
9,Comunicação,"-1,16","-0,05",2015-mar


In [42]:
df_final_12_8.to_excel("df_shapes/df_12_8.xlsx", index=False)

## Shape (4,4) !!

In [68]:
arquivos_shape_4_4 = shape_to_files.get((4,4), [])

fname = arquivos_shape_4_4[0]

dfs = tabula.read_pdf(fname, pages="all", multiple_tables=True)

print("Número de tabelas extraídas:", len(dfs))
for i, df in enumerate(dfs[:2]):
    print("\n--- Tabela", i, "---")
    #teste = df
    display(df)

Número de tabelas extraídas: 48

--- Tabela 0 ---


,Variação (%),Impacto (p.p.),Unnamed: 0,Unnamed: 1
0,Novembro,Dezembro,Novembro,Dezembro
1,NaN,NaN,NaN,NaN
2,"0,51","0,78","0,51\r0,19\r0,10\r0,00\r0,03\r0,08\r0,05\r0,05...",NaN
3,"0,77\r0,69\r-0,04\r0,39\r0,43\r0,42\r0,48\r0,2...","1,08\r0,51\r0,00\r0,85\r1,38\r0,47\r0,70\r0,07...",NaN,NaN



--- Tabela 1 ---


,Variação mensal (%),Unnamed: 0
0,Novembro,Dezembro
1,"38,71\r0,14\r-2,67\r3,66\r1,07\r1,56\r-5,34\r3...","13,77\r12,62\r7,49\r7,40\r5,18\r4,80\r3,84\r3,..."


## Shape (5,5) !!

In [67]:
arquivos_shape_5_5 = shape_to_files.get((5,5), [])

fname = arquivos_shape_5_5[0]

dfs = tabula.read_pdf(fname, pages="all", multiple_tables=True)

print("Número de tabelas extraídas:", len(dfs))
for i, df in enumerate(dfs[:2]):
    print("\n--- Tabela", i, "---")
    #teste = df
    display(df)

Número de tabelas extraídas: 16

--- Tabela 0 ---


,Unnamed: 0,Variação (%),Impacto (p.p.),Unnamed: 1,Unnamed: 2
0,NaN,Janeiro,Fevereiro,Janeiro,Fevereiro
1,NaN,NaN,NaN,NaN,NaN
2,Índice Geral,"0,38\r0,35\r0,17\r-0,10\r-0,36\r0,77\r0,55\r0,...","0,33\r-0,45\r0,24\r0,18\r-0,13\r0,24\r0,65\r0,...","0,38\r0,09\r0,03\r0,00\r-0,02\r0,14\r0,06\r0,0...",NaN
3,Habitação\rArtigos de Residência\rVestuário,NaN,NaN,NaN,NaN
4,Saúde e Cuidados Pessoais\rDespesas Pessoais\r...,NaN,NaN,NaN,NaN



--- Tabela 1 ---


,Unnamed: 0,Variação\r(%)
0,NaN,Fevereiro
1,Salvador\rRecife\rBelo Horizonte\rRio de Janei...,NaN
2,NaN,NaN
3,Brasil,NaN


## Shape (4,5)

In [71]:
arquivos_shape_4_5 = shape_to_files.get((4,5), [])

fname = arquivos_shape_4_5[0]

dfs = tabula.read_pdf(fname, pages="all", multiple_tables=True)

print("Número de tabelas extraídas:", len(dfs))
for i, df in enumerate(dfs[:2]):
    print("\n--- Tabela", i, "---")
    #teste = df
    display(df)

Número de tabelas extraídas: 16

--- Tabela 0 ---


,Grupo,Variação (%),Impacto (P.P),Unnamed: 0,Unnamed: 1
0,NaN,Junho,Julho,Junho,Julho
1,NaN,NaN,NaN,NaN,NaN
2,Índice Geral,"0,08","0,43","0,08",NaN
3,Alimentação e Bebidas\rHabitação\rArtigos de R...,"0,68\r0,28\r-0,03\r0,39\r-1,18\r0,38\r0,47\r0,...","0,91\r0,54\r-0,01\r0,04\r-0,03\r0,36\r0,91\r0,...","0,16\r0,04\r0,00\r0,03\r-0,24\r0,04\r0,05\r0,0...","0,21\r0,08\r0,00\r0,00\r-0,01\r0,04\r0,09\r0,0..."



--- Tabela 1 ---


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Variação (%)
0,NaN,Item,NaN,NaN
1,NaN,Junho,NaN,Julho Ano 12 meses
2,Tomate,"11,45",NaN,"50,33 48,33 43,29"
3,Cenoura,"5,80",NaN,"17,81 46,83 36,88"
4,Alho,"5,92",NaN,"12,27 32,75 -20,42"
5,Feijão-preto,"3,48",NaN,"6,12 38,33 41,58"
6,Hortaliças,"2,92",NaN,"4,68 24,18 19,05"
7,Feijão-mulatinho,"7,90",NaN,"4,38 81,40 84,38"
8,Ovo,"0,80",NaN,"2,63 8,47 7,91"
9,Lanche,"0,63",NaN,"2,62 5,95 10,90"


## Shape (2, 4) - !!

In [63]:
arquivos_shape_2_4 = shape_to_files.get((2,4), [])

In [64]:
fname = arquivos_shape_2_4[0]

dfs = tabula.read_pdf(fname, pages="all", multiple_tables=True)

print("Número de tabelas extraídas:", len(dfs))
for i, df in enumerate(dfs[:2]):
    print("\n--- Tabela", i, "---")
    #teste = df
    display(df)

Número de tabelas extraídas: 11

--- Tabela 0 ---


,Variação (%),Impacto (p.p.),Unnamed: 0,Unnamed: 1
0,Fevereiro,Março,Fevereiro,Março
1,"0,25\r0,11\r-0,39\r-0,08\r-0,73\r-0,23\r0,73\r...","0,07\r1,13\r0,13\r-1,08\r0,21\r-0,90\r0,21\r-0...","0,25\r0,02\r-0,06\r0,00\r-0,03\r-0,05\r0,10\r0...",NaN



--- Tabela 1 ---


,Unnamed: 0,Peso,Variação (%),Variação Acumulada (%)
0,Região,Regional,NaN,NaN
1,NaN,(%),NaN,NaN
2,NaN,NaN,Fevereiro Março,Ano 12 meses
3,Campo Grande,"1,57","0,42 0,56","1,11 4,33"
4,Rio de Janeiro,"9,43","-0,02 0,46","0,50 2,71"
5,Aracaju,"1,03","0,66 0,41","1,47 3,52"
6,São Luís,"1,62","0,18 0,37","0,36 2,72"
7,Recife,"3,92","0,38 0,31","0,99 2,99"
8,Fortaleza,"3,23","0,80 0,21","1,29 4,39"
9,Salvador,"5,99","0,16 0,17","0,67 3,27"


## Shape (3,4)

In [59]:
arquivos_shape_3_4 = shape_to_files.get((3,4), [])

In [60]:
fname = arquivos_shape_3_4[0]

dfs = tabula.read_pdf(fname, pages="all", multiple_tables=True)

print("Número de tabelas extraídas:", len(dfs))
for i, df in enumerate(dfs[:2]):
    print("\n--- Tabela", i, "---")
    #teste = df
    display(df)

Número de tabelas extraídas: 18

--- Tabela 0 ---


,"COMENTÁRIOS\rAbril de 2018\r\r1.\rÍNDICE NACIONAL DE PREÇOS AO CONSUMIDOR AMPLO - IPCA\r\rO Índice Nacional de Preços ao Consumidor Amplo - IPCA do mês de abril apresentou\rvariação de 0,22% e ficou 0,13 ponto percentual (p.p.) acima da taxa de 0,09% registrada em março. O\racumuladonoano,queregistrou0,92%,apresentouomenornívelparaummêsdeabrildesdea\rimplantação do Plano Real. Na ótica dos últimos doze meses, o índice ficou em 2,76%, enquanto havia\rregistrado 2,68% nos 12 meses imediatamente anteriores. Em abril de 2017, a taxa atingiu 0,14%.\r\rPara cálculo do índice do mês foram comparados os preços coletados no período de 30 de março\ra 27 de abril de 2018 (referência) com os preços vigentes no período de 02 de março a 29 de março de\r2018 (base).\r\rDentre os nove grupos de produtos e serviços pesquisados, apenas Comunicação apresentou\rdeflação em abril, com variação de -0,07%. Transportes, com variação de 0,00% mostrou, na média,\restabilidade nos preços de março para abril. Já os demais grupos vieram com alta, variando de 0,08% a\r0,91%, conforme pode ser observado na tabela abaixo.\r\rVariação (%)Impacto (p.p.)\rGrupo\rMarçoAbrilMarçoAbril\rÍndice Geral0,090,220,090,22\rAlimentação e Bebidas0,070,090,020,02\rHabitação0,190,170,030,03\rArtigos de Residência0,080,220,000,01\rVestuário0,330,620,020,04\rTransportes-0,250,00-0,050,00\rSaúde e Cuidados Pessoais0,480,910,060,11\rDespesas Pessoais0,050,120,010,01\rEducação0,280,080,010,00\rComunicação-0,33-0,07-0,010,00\r\rO grupo Saúde e cuidados pessoais apresentou, além da maior variação no mês (0,91%), a\rmaior contribuição (0,11 p.p.), respondendo por metade do IPCA de abril. Sobressaem os remédios\r- 8 -",Unnamed: 0,Unnamed: 1,Unnamed: 2
0,Variação (%),Impacto (p.p.),NaN,NaN
1,Março,Abril,Março,Abril
2,"0,09\r0,07\r0,19\r0,08\r0,33\r-0,25\r0,48\r0,0...","0,22\r0,09\r0,17\r0,22\r0,62\r0,00\r0,91\r0,12...","0,09\r0,02\r0,03\r0,00\r0,02\r-0,05\r0,06\r0,0...",NaN



--- Tabela 1 ---


,"COMENTÁRIOS\rAbril de 2018\r\r1.\rÍNDICE NACIONAL DE PREÇOS AO CONSUMIDOR AMPLO - IPCA\r\rO Índice Nacional de Preços ao Consumidor Amplo - IPCA do mês de abril apresentou\rvariação de 0,22% e ficou 0,13 ponto percentual (p.p.) acima da taxa de 0,09% registrada em março. O\racumuladonoano,queregistrou0,92%,apresentouomenornívelparaummêsdeabrildesdea\rimplantação do Plano Real. Na ótica dos últimos doze meses, o índice ficou em 2,76%, enquanto havia\rregistrado 2,68% nos 12 meses imediatamente anteriores. Em abril de 2017, a taxa atingiu 0,14%.\r\rPara cálculo do índice do mês foram comparados os preços coletados no período de 30 de março\ra 27 de abril de 2018 (referência) com os preços vigentes no período de 02 de março a 29 de março de\r2018 (base).\r\rDentre os nove grupos de produtos e serviços pesquisados, apenas Comunicação apresentou\rdeflação em abril, com variação de -0,07%. Transportes, com variação de 0,00% mostrou, na média,\restabilidade nos preços de março para abril. Já os demais grupos vieram com alta, variando de 0,08% a\r0,91%, conforme pode ser observado na tabela abaixo.\r\rVariação (%)Impacto (p.p.)\rGrupo\rMarçoAbrilMarçoAbril\rÍndice Geral0,090,220,090,22\rAlimentação e Bebidas0,070,090,020,02\rHabitação0,190,170,030,03\rArtigos de Residência0,080,220,000,01\rVestuário0,330,620,020,04\rTransportes-0,250,00-0,050,00\rSaúde e Cuidados Pessoais0,480,910,060,11\rDespesas Pessoais0,050,120,010,01\rEducação0,280,080,010,00\rComunicação-0,33-0,07-0,010,00\r\rO grupo Saúde e cuidados pessoais apresentou, além da maior variação no mês (0,91%), a\rmaior contribuição (0,11 p.p.), respondendo por metade do IPCA de abril. Sobressaem os remédios\r- 8 -",Unnamed: 0,Unnamed: 1,Unnamed: 2
0,Variação (%),Impacto (p.p.),NaN,NaN
1,Março,Abril,Março,Abril
2,"0,09\r0,07\r0,19\r0,08\r0,33\r-0,25\r0,48\r0,0...","0,22\r0,09\r0,17\r0,22\r0,62\r0,00\r0,91\r0,12...","0,09\r0,02\r0,03\r0,00\r0,02\r-0,05\r0,06\r0,0...",NaN


## Concatenando as tabelas geradas automatizadas e manualmente

In [ ]:
df_manual = pd.read_excel("shapes_faltantes.xlsx")
# df_final_2_5 = pd.read_excel("df_shapes/df_2_5.xlsx")
# df_final_12_3 = pd.read_excel("df_shapes/df_12_3.xlsx")
# df_final_12_4 = pd.read_excel("df_shapes/df_12_4.xlsx")
# df_final_12_8 = pd.read_excel("df_shapes/df_12_8.xlsx")
# df_final_14_3 = pd.read_excel("df_shapes/df_14_3.xlsx")

In [128]:
df_manual.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 0 to 259
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Setor           260 non-null    object 
 1   variação (%)    260 non-null    float64
 2   impacto (p.p.)  260 non-null    float64
 3   ano_mes         260 non-null    object 
dtypes: float64(2), object(2)
memory usage: 8.3+ KB


In [129]:
len(df_manual) + len(df_final_2_5) + len(df_final_12_3) + len(df_final_12_4) + len(df_final_12_8) + len(df_final_14_3)

1080

In [121]:
df_manual.head(1)

,Setor,variação (%),impacto (p.p.),ano_mes
0,Índice Geral,0.78,0.78,2014-dez


In [130]:
df_final_2_5.head(1)
df_final_2_5 = df_final_2_5.rename(columns = {'Variação (%)': 'variação (%)', 'Impacto (p.p.)': 'impacto (p.p.)'})

In [131]:
df_final_12_3.head(1)
df_final_12_3 = df_final_12_3.drop(columns=['Variação (%)', 'Impacto (p.p.)'])

In [132]:
df_final_12_4.head(1)
df_final_12_4 = df_final_12_4.drop(columns=['Variação (%)', 'Impacto (p.p.)'])

In [133]:
df_final_12_8.head(1)

,Setor,variação (%),impacto (p.p.),ano_mes
0,Índice Geral,"1,32","1,32",2015-mar


In [134]:
df_final_14_3.head(1)
df_final_14_3 = df_final_14_3.drop(columns=['Variação (%)', 'Impacto (p.p.)'])

In [135]:
df_concat = pd.concat([df_final_2_5, df_final_12_3, df_final_12_4, df_final_12_8, df_final_14_3], ignore_index=True)

In [136]:
df_concat

,Setor,variação (%),impacto (p.p.),ano_mes
0,Índice Geral,"0,86","0,86",2013-jan
1,Alimentação e Bebidas,"1,99","0,48",2013-jan
2,Habitação,"-0,20","-0,03",2013-jan
3,Artigos de Residência,"1,15","0,05",2013-jan
4,Vestuário,"-0,53","-0,04",2013-jan
...,...,...,...,...
815,Transportes,"1,36","0,27",2020-dez
816,Saúde e cuidados pessoais,"0,40","0,05",2020-dez
817,Despesas pessoais,"0,65","0,07",2020-dez
818,Educação,"0,48","0,03",2020-dez


In [137]:
df_concat['variação (%)'] = df_concat['variação (%)'].str.replace(',', '.').astype(float)
df_concat['impacto (p.p.)'] = df_concat['impacto (p.p.)'].str.replace(',', '.').astype(float)

In [138]:
df_ipca_setor = pd.concat([df_concat, df_manual], ignore_index=True)

In [139]:
df_ipca_setor

,Setor,variação (%),impacto (p.p.),ano_mes
0,Índice Geral,0.86,0.86,2013-jan
1,Alimentação e Bebidas,1.99,0.48,2013-jan
2,Habitação,-0.20,-0.03,2013-jan
3,Artigos de Residência,1.15,0.05,2013-jan
4,Vestuário,-0.53,-0.04,2013-jan
...,...,...,...,...
1075,Transportes,0.60,0.10,2012-jan
1076,Saúde e Cuidados Pessoais,0.30,0.03,2012-jan
1077,Despesas Pessoais,0.71,0.07,2012-jan
1078,Educação,0.39,0.02,2012-jan


In [140]:
df_ipca_setor.isnull().sum()

Setor             0
variação (%)      0
impacto (p.p.)    0
ano_mes           0
dtype: int64

## Gerar IPCA por Setor por ano

In [141]:
df_ipca_setor.Setor.unique()

array(['Índice Geral', 'Alimentação e Bebidas', 'Habitação',
       'Artigos de Residência', 'Vestuário', 'Transportes',
       'Saúde e Cuidados Pessoais', 'Despesas Pessoais', 'Educação',
       'Comunicação', 'Alimentação e bebidas', 'Artigos de residência',
       'Saúde e cuidados pessoais', 'Despesas pessoais'], dtype=object)

In [142]:
df_ipca_setor['Setor'] = df_ipca_setor['Setor'].str.lower().str.strip()

In [143]:
df_ipca_setor.Setor.unique()

array(['índice geral', 'alimentação e bebidas', 'habitação',
       'artigos de residência', 'vestuário', 'transportes',
       'saúde e cuidados pessoais', 'despesas pessoais', 'educação',
       'comunicação'], dtype=object)

In [144]:
df_pivot = df_ipca_setor.pivot(
    index='ano_mes',
    columns='Setor',
    values='variação (%)'
)

# Achatar as colunas
df_pivot.columns = df_pivot.columns.get_level_values(0)  # ou get_level_values(1) dependendo do caso
# ou de forma mais robusta:
# df_pivot.columns = [col[1] if isinstance(col, tuple) else col for col in df_pivot.columns]

# Trazer 'ano_mes' de volta como coluna normal
df_pivot = df_pivot.reset_index()

In [145]:
df_pivot

Setor,ano_mes,alimentação e bebidas,artigos de residência,comunicação,despesas pessoais,educação,habitação,saúde e cuidados pessoais,transportes,vestuário,índice geral
0,2012-abr,0.51,-0.79,0.46,2.23,0.04,0.80,0.96,0.10,0.98,0.64
1,2012-ago,0.88,0.40,-0.01,0.42,0.51,0.22,0.53,0.06,0.19,0.41
2,2012-dez,1.03,0.27,0.03,1.60,0.19,0.63,0.40,0.75,1.11,0.79
3,2012-fev,0.19,0.06,-0.17,0.88,5.62,0.60,0.70,-0.33,-0.23,0.45
4,2012-jan,0.86,0.16,0.21,0.71,0.39,0.53,0.30,0.60,0.07,0.56
...,...,...,...,...,...,...,...,...,...,...,...
103,2020-mai,0.24,0.58,0.24,-0.04,0.02,-0.25,-0.10,-1.90,-0.58,-0.38
104,2020-mar,1.13,-1.08,0.04,-0.23,0.59,0.13,0.21,-0.90,0.21,0.07
105,2020-nov,2.54,0.86,0.29,0.01,-0.02,0.44,-0.13,1.33,0.07,0.89
106,2020-out,1.93,1.53,0.21,0.19,-0.04,0.36,0.28,1.19,1.11,0.86


In [146]:
df_ipca_setor.ano_mes.unique()

array(['2013-jan', '2012-mar', '2012-abr', '2012-jun', '2012-set',
       '2012-nov', '2020-jan', '2020-fev', '2020-abr', '2019-ago',
       '2019-set', '2019-out', '2019-nov', '2016-jan', '2016-abr',
       '2015-abr', '2015-jun', '2015-out', '2014-jan', '2014-fev',
       '2014-mar', '2014-abr', '2013-fev', '2013-mar', '2013-abr',
       '2013-mai', '2013-jun', '2013-jul', '2013-ago', '2013-set',
       '2013-nov', '2013-dez', '2012-mai', '2012-ago', '2012-out',
       '2012-dez', '2020-mai', '2019-jan', '2019-fev', '2019-mar',
       '2019-abr', '2019-mai', '2019-jun', '2019-jul', '2017-jan',
       '2017-mar', '2017-abr', '2017-mai', '2017-jun', '2017-jul',
       '2017-ago', '2017-set', '2017-out', '2017-nov', '2017-dez',
       '2016-mar', '2016-mai', '2016-jul', '2016-ago', '2016-set',
       '2016-out', '2016-nov', '2016-dez', '2015-jan', '2015-mai',
       '2015-jul', '2015-ago', '2015-nov', '2014-mai', '2014-jun',
       '2014-jul', '2014-ago', '2014-set', '2014-out', '2014-n

In [147]:
df_pivot.ano_mes.unique()

array(['2012-abr', '2012-ago', '2012-dez', '2012-fev', '2012-jan',
       '2012-jul', '2012-jun', '2012-mai', '2012-mar', '2012-nov',
       '2012-out', '2012-set', '2013-abr', '2013-ago', '2013-dez',
       '2013-fev', '2013-jan', '2013-jul', '2013-jun', '2013-mai',
       '2013-mar', '2013-nov', '2013-out', '2013-set', '2014-abr',
       '2014-ago', '2014-dez', '2014-fev', '2014-jan', '2014-jul',
       '2014-jun', '2014-mai', '2014-mar', '2014-nov', '2014-out',
       '2014-set', '2015-abr', '2015-ago', '2015-dez', '2015-fev',
       '2015-jan', '2015-jul', '2015-jun', '2015-mai', '2015-mar',
       '2015-nov', '2015-out', '2015-set', '2016-abr', '2016-ago',
       '2016-dez', '2016-fev', '2016-jan', '2016-jul', '2016-jun',
       '2016-mai', '2016-mar', '2016-nov', '2016-out', '2016-set',
       '2017-abr', '2017-ago', '2017-dez', '2017-fev', '2017-jan',
       '2017-jul', '2017-jun', '2017-mai', '2017-mar', '2017-nov',
       '2017-out', '2017-set', '2018-abr', '2018-ago', '2018-d

In [148]:
df_pivot.columns

Index(['ano_mes', 'alimentação e bebidas', 'artigos de residência',
       'comunicação', 'despesas pessoais', 'educação', 'habitação',
       'saúde e cuidados pessoais', 'transportes', 'vestuário',
       'índice geral'],
      dtype='object', name='Setor')

In [153]:
import locale
locale.setlocale(locale.LC_TIME, 'pt_BR.UTF-8')


# Converter ano_mes para datetime
df_pivot['ano_mes'] = pd.to_datetime(df_pivot['ano_mes'], format='%Y-%b')

# Extrair o ano
df_pivot['ano'] = df_pivot['ano_mes'].dt.year

# Selecionar apenas colunas numéricas dos setores
setores = df_pivot.columns.difference(['ano_mes', 'ano', 'Setor'])

# Calcular IPCA anualizado por setor
ipca_anualizado = (
    df_pivot
    .groupby('ano')[setores]
    .apply(lambda x: ((1 + x/100).prod() - 1) * 100)
    .reset_index()  # <-- transforma o ano de índice em coluna
)

# Ordenar pelo ano (opcional, mas deixa organizado)
ipca_anualizado = ipca_anualizado.sort_values('ano').reset_index(drop=True)

In [154]:
ipca_anualizado

Setor,ano,alimentação e bebidas,artigos de residência,comunicação,despesas pessoais,educação,habitação,saúde e cuidados pessoais,transportes,vestuário,índice geral
0,2012,9.862794,0.846219,0.759539,10.156268,7.792614,6.811951,5.943856,0.374033,5.800558,5.838569
1,2013,8.477348,7.116327,1.506580,8.398714,7.922577,3.409809,6.947191,3.295580,5.373638,5.910818
2,2014,8.031129,5.496404,-1.509972,8.305893,8.452358,8.803604,6.970412,3.758790,3.649534,6.407617
3,2015,12.008131,5.382886,2.096074,9.510231,9.248743,18.336453,9.202803,10.167379,4.469602,10.673498
4,2016,8.610738,3.412772,1.270565,8.009707,8.865806,2.839685,11.045369,4.236124,3.539347,6.288055
5,2017,-1.867985,-1.492399,1.753682,4.384830,7.081174,6.277734,6.514752,4.097180,2.886157,2.947350
6,2018,4.030782,3.759616,-0.080826,2.989517,5.310920,4.724189,3.951694,4.202469,0.602078,3.745482
7,2019,6.364091,-0.377431,1.081717,4.669742,4.757728,3.895636,5.410017,3.582840,0.738737,4.306040
8,2020,14.113456,5.996319,3.428847,1.021320,1.134802,5.274873,1.501329,1.031892,-1.113577,4.517342


In [155]:
ipca_anualizado = ipca_anualizado.rename(columns = {
    'alimentação e bebidas': 'ipca_alimentos_bebidas',
    'artigos de residência': 'ipca_artigos_residencia',
    'comunicação': 'ipca_comunicacao',
    'habitação': 'ipca_habitacao',
    'transportes': 'ipca_transportes',
    'despesas pessoais': 'ipca_despesas_pessoais',
    'educação': 'ipca_educacao',
    'saúde e cuidados pessoais': 'ipca_saude_cuidados_pessoais',
    'vestuário': 'ipca_vestuario',
    'índice geral': 'ipca_geral'

})

In [156]:
ipca_anualizado

Setor,ano,ipca_alimentos_bebidas,ipca_artigos_residencia,ipca_comunicacao,ipca_despesas_pessoais,ipca_educacao,ipca_habitacao,ipca_saude_cuidados_pessoais,ipca_transportes,ipca_vestuario,ipca_geral
0,2012,9.862794,0.846219,0.759539,10.156268,7.792614,6.811951,5.943856,0.374033,5.800558,5.838569
1,2013,8.477348,7.116327,1.506580,8.398714,7.922577,3.409809,6.947191,3.295580,5.373638,5.910818
2,2014,8.031129,5.496404,-1.509972,8.305893,8.452358,8.803604,6.970412,3.758790,3.649534,6.407617
3,2015,12.008131,5.382886,2.096074,9.510231,9.248743,18.336453,9.202803,10.167379,4.469602,10.673498
4,2016,8.610738,3.412772,1.270565,8.009707,8.865806,2.839685,11.045369,4.236124,3.539347,6.288055
5,2017,-1.867985,-1.492399,1.753682,4.384830,7.081174,6.277734,6.514752,4.097180,2.886157,2.947350
6,2018,4.030782,3.759616,-0.080826,2.989517,5.310920,4.724189,3.951694,4.202469,0.602078,3.745482
7,2019,6.364091,-0.377431,1.081717,4.669742,4.757728,3.895636,5.410017,3.582840,0.738737,4.306040
8,2020,14.113456,5.996319,3.428847,1.021320,1.134802,5.274873,1.501329,1.031892,-1.113577,4.517342


In [157]:
ipca_anualizado.to_excel("ipca_setor_anualizado.xlsx", index=False)